In [46]:
import numpy as np
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

texts_true = [
    "Цей фільм був надзвичайно захоплюючим і цікавим. Я отримав неймовірне задоволення від перегляду!",
    "Дуже задоволений роботою цієї компанії. Вони надзвичайні!",
    "Ця книга - справжнє диво літератури. Вона вражає своєю глибиною та мудрістю на кожній сторінці."
]

texts_false = [
    "Цей ресторан залишив мене розчарованим. Їжа була несмачною, обслуговування погане, а атмосфера нудна.",
    "Жахливе обслуговування! Не раджу нікому!",
    "Цей фільм - повний провал. Сюжет нудний, персонажі непривабливі, а акторська гра слабка."
]

# Об'єднання текстів
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]


model = Sequential()
model.add(Embedding(maxWordsCount, 128))
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh')))
model.add(Bidirectional(LSTM(64, activation='tanh')))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adamax(learning_rate=0.001))

history = model.fit(X, Y, batch_size=32, epochs=50)

def predict_sentiment(text):
    data = tokenizer.texts_to_sequences([text.lower()])
    data_pad = pad_sequences(data, maxlen=max_text_len)
    prediction = model.predict(data_pad)
    if prediction[0][0] > prediction[0][1]:
        return "Позитивний"
    else:
        return "Негативний"

positive = "Це найкращий фільм, який я коли-небудь бачив!"
negative = "Жахлива книга, я не рекомендую її."


3 3 6
[[ 0  6  7  8  3  9 10  1 11 12]
 [ 0  4 13  5 14 15 16  1 17 18]
 [ 0 19 20 21 22 23 24 25  2 26]
 [ 3 27  4 28  2 29 30 31  2 32]
 [ 0  5 33 34 35 36 37  1 38 39]
 [ 0  0  0 40 41 42 43 44  1 45]]
(6, 10) (6, 2)
Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.3333 - loss: 0.6955
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 0.6766
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 0.6587
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.6408
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.6218
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.6010
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.5779
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.5523
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.5239
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━

In [47]:
print(predict_sentiment(positive))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
Позитивний


In [48]:
print(predict_sentiment(negative))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Негативний
